<a href="https://colab.research.google.com/github/yuribp0107/Congreso_Tecnologos_medicos_2023/blob/main/simulador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML
HTML('''
<style>
  /* Oculta todas las celdas de código */
  div.input { display: none !important; }
</style>
''')


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets, Layout
from IPython.display import display

# Constantes HU y gradiente completo
hu_min, hu_max = -1000, 1000
scale_h = 300
full_grad = np.linspace(hu_max, hu_min, scale_h).reshape((scale_h,1))

def generate_synthetic_ct(shape=(200,200)):
    img = np.full(shape, -1000, dtype=float)
    yy, xx = np.indices(shape)
    center = np.array(shape)/2
    a, b = shape[0]*0.4, shape[1]*0.4

    # Sustancia blanca
    mask_wm = ((xx-center[1])**2/(a*0.8)**2 +
               (yy-center[0])**2/(b*0.8)**2) <= 1
    # Sustancia gris
    mask_gm = ((xx-center[1])**2/a**2 +
               (yy-center[0])**2/b**2) <= 1
    mask_gm &= ~mask_wm

    # Ventrículos
    vent1 = ((xx-center[1]-0.2*a)**2/(0.15*a)**2 +
             (yy-center[0])**2/(0.4*b)**2) <= 1
    vent2 = ((xx-center[1]+0.2*a)**2/(0.15*a)**2 +
             (yy-center[0])**2/(0.4*b)**2) <= 1
    mask_vent = vent1 | vent2

    # 1) Hematoma desplazado a la derecha
    h_cy, h_cx = center[0], center[1] + 0.8*a
    mask_hema = ((xx - h_cx)**2/(0.1*a)**2 +
                 (yy - h_cy)**2/(0.1*b)**2) <= 1

    # 2) Edema en posición distinta (por ejemplo, arriba-izquierda)
    e_cy, e_cx = center[0] - 0.5*b, center[1] - 0.6*a
    mask_edema = ((xx - e_cx)**2/(0.15*a)**2 +
                  (yy - e_cy)**2/(0.15*b)**2) <= 1

    # Cráneo
    outer = ((xx-center[1])**2/(a+5)**2 +
             (yy-center[0])**2/(b+5)**2) <= 1
    mask_skull = outer & ~mask_gm & ~mask_wm

    # Grasa periférica
    fat_outer = ((xx-center[1])**2/(a+15)**2 +
                 (yy-center[0])**2/(b+15)**2) <= 1
    mask_fat = fat_outer & ~outer

    # Medio de contraste
    rw, rh = 0.3*a, 0.1*b
    x0 = center[1] - 0.7*a
    y0 = center[0] + 0.4*b
    mask_contrast = ((xx >= x0) & (xx <= x0+rw) &
                     (yy >= y0) & (yy <= y0+rh))

    img[mask_gm]       =   40
    img[mask_wm]       =   20
    img[mask_vent]     =    0
    img[mask_edema]    =   10
    img[mask_hema]     =   90
    img[mask_skull]    = 1000
    img[mask_fat]      = -100
    img[mask_contrast] =  300

    return img

def hu_to_pixel(h):
    return (hu_max - h)/(hu_max - hu_min)*(scale_h - 1)

def apply_window(img, wc, ww):
    low, up = wc - ww/2, wc + ww/2
    win = np.clip(img, low, up)
    return (win - low)/(up - low)

# Precomputar TC
img_hu = generate_synthetic_ct()

def plot_synthetic_ct(wc, ww):
    """Figura 1: TC sintético con nivel/ancho de ventana."""
    plt.figure(figsize=(5,5))
    img_win = apply_window(img_hu, wc, ww)
    plt.imshow(img_win, cmap='gray', origin='lower')
    plt.axis('off')
    plt.title(f'TC Sintético\nWC={wc}, WW={ww}', pad=10)
    plt.show()

    """Figura 2: Escalas HU (completa y windowed)."""
    lower, upper = wc - ww/2, wc + ww/2

    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8,5))
    plt.subplots_adjust(wspace=0.4)

    # Escala completa con líneas WC/WW
    ax0.imshow(full_grad, cmap='gray', origin='upper', aspect='auto',
               vmin=hu_min, vmax=hu_max)
    ax0.set_xticks([])
    ticks0 = np.linspace(0, scale_h-1, 5)
    ax0.set_yticks(ticks0)
    ax0.set_yticklabels(np.linspace(hu_max, hu_min, 5, dtype=int))
    ax0.hlines(hu_to_pixel(wc),   -0.5, 0.5, color='orange', lw=2, label='Centro (WC)')
    ax0.hlines(hu_to_pixel(lower), -0.5, 0.5, color='red', ls='--', label='Límite inf.')
    ax0.hlines(hu_to_pixel(upper), -0.5, 0.5, color='red', ls='-.', label='Límite sup.')
    ax0.legend(loc='upper right', fontsize='small')
    ax0.set_title('Escala HU Base')

    # Escala windowed
    dyn_grad = np.linspace(upper, lower, scale_h).reshape((scale_h,1))
    norm = (dyn_grad - lower)/(upper - lower)
    ax1.imshow(norm, cmap='gray', origin='upper', aspect='auto', vmin=0, vmax=1)
    ax1.set_xticks([])
    ticks1 = np.linspace(0, scale_h-1, 5)
    ax1.set_yticks(ticks1)
    ax1.set_yticklabels(np.linspace(int(upper), int(lower), 5, dtype=int))
    ax1.set_title('Escala HU')
    plt.show()

# Widgets interactivos
wc_slider = widgets.IntSlider(value=35, min=-200, max=600, step=5,
                              description='WC', layout=Layout(width='400px'))
ww_slider = widgets.IntSlider(value=100, min=1, max=2000, step=10,
                              description='WW', layout=Layout(width='400px'))

interactive_plot = interactive(plot_synthetic_ct, wc=wc_slider, ww=ww_slider)
display(interactive_plot)  # Te recomiendo no modificar nada


interactive(children=(IntSlider(value=35, description='WC', layout=Layout(width='400px'), max=600, min=-200, s…